[View in Colaboratory](https://colab.research.google.com/github/bell-one/tf_colab/blob/master/YELLOW_LSTM_deep_softmax_tf.ipynb)

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [0]:
sample = ("Look at the stars Look how they shine for you And everything you do"
"Yeah they were all yellow I came along I wrote a song for you"
"And all the things you do And it was called Yellow"
"So then I took my turn Oh what a thing to have done And it was all yellow"
"Your skin Oh yeah, your skin and bones Turn into something beautiful You know, you know I love you so You know I love you so"          
"I swam across I jumped across for you Oh what a thing to do Cause you were all yellow"          
"I drew a line I drew a line for you Oh what a thing to do And it was all yellow"          
"Your skin Oh yeah your skin and bones Turn into something beautiful And you know For you I'd bleed myself dry For you I'd bleed myself dry"          
"It's true Look how they shine for you Look how they shine for you Look how they shine for Look how they shine for you Look how they shine for you Look how they shine"          
"Look at the stars Look how they shine for you And all the things that you do")
          
char_set = list(set(sample))
char_dic = {w: i for i, w in enumerate(char_set)}

In [0]:
# hyper parameters
dic_size = len(char_set)  # RNN input size (one hot size)
hidden_size = len(char_set)  # RNN output size
num_classes = len(char_set)  # final output size (RNN or softmax, etc.)
learning_rate = 0.1
seq_length = 10  # number of lstm rollings (unit #)

dataX = []
dataY = []

for i in range(0, len(sample) - seq_length):
  x_str = sample[i:i + seq_length]
  y_str = sample[i+1:i+seq_length+1]
  
  x = [char_dic[c] for c in x_str]
  y = [char_dic[c] for c in x_str]
  
  dataX.append(x)
  dataY.append(y)


batch_size = len(dataX)

In [0]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])
X_one_hot = tf.one_hot(X, num_classes)

In [0]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell]*2, state_is_tuple=True )
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state = initial_state, dtype=tf.float32)



X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
softmax_w = tf.get_variable("softw",[hidden_size, num_classes])
softmax_b = tf.get_variable("softb",[num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w)+softmax_b
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

weights = tf.ones([batch_size,seq_length])
seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets =Y, weights=weights)

loss = tf.reduce_mean(seq_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)


In [6]:
sess.run(tf.global_variables_initializer())
for i in range(300):
    _, l, results = sess.run(
        [train, loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
    if i % 20 == 0:
        print("epoch:",i," loss:",l)


# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

epoch: 0  loss: 3.5997207
epoch: 20  loss: 1.1827987
epoch: 40  loss: 0.0340239
epoch: 60  loss: 0.0020264504
epoch: 80  loss: 0.0007583974
epoch: 100  loss: 0.0005081338
epoch: 120  loss: 0.00040097814
epoch: 140  loss: 0.0003329739
epoch: 160  loss: 0.000282697
epoch: 180  loss: 0.0002434888
epoch: 200  loss: 0.00021239508
epoch: 220  loss: 0.00018741773
epoch: 240  loss: 0.00016704448
epoch: 260  loss: 0.00015019719
epoch: 280  loss: 0.00013605224
Look at the stars Look how they shine for you And everything you doYeah they were all yellow I came along I wrote a song for youAnd all the things you do And it was called YellowSo then I took my turn Oh what a thing to have done And it was all yellowYour skin Oh yeah, your skin and bones Turn into something beautiful You know, you know I love you so You know I love you soI swam across I jumped across for you Oh what a thing to do Cause you were all yellowI drew a line I drew a line for you Oh what a thing to do And it was all yellowYour s